# 2 - BBS with ECDSA Holder Binding

While BBS allows to present parts of the Verifiable Credential in an anonymous
and unlinkable manner, there are limits.
The [RFC](https://datatracker.ietf.org/doc/draft-irtf-cfrg-bbs-signatures/) only
supports selective disclosure.
This works for fields which have a big anonymity set, like the NPA, or the canton of
residence.
However, for more linkable fields, like the birthdate, full name, the anonymity set
gets very small and can drop to 1.
An anonymity set of 1 means that there is only one VC with that field, so it can
be linked between usages.

To achieve holder binding, it is necessary to sign each presentation of a VC
with the private key of the device.
For the verification part, the verifier usually needs the public key.
However, this public key is unique.
Even worse, the holder binding needs to be done for every presentation of the VC,
so all presentations can be linked!

One way to solve this is to create many VCs for a user, each VC with a corresponding
public key created by the Secure Element of the wallet, e.g., the phone.
This is not an ideal solution for the following reasons: 

- it increases load on the server who has to issue VCs on a regular basis
- to increase privacy, these must be renewed, even if the VCs are not used
- some secure elements have a limit of how many private keys they can store

Several other solutions have been proposed so far:

- BBS# from Orange - which is missing a scientific paper
[Presentation](https://csrc.nist.gov/csrc/media/presentations/2024/wpec2024-3b3/images-media/wpec2024-3b3-slides-antoine-jacques--BBS-sharp-eIDAS2.pdf)
- Google ZKP, presented at Funke 24 - but in June '25 there is still no code available
- Ubique proposed a solution based on [ZKAttest](https://eprint.iacr.org/2021/1183) - with code available, and based on a peer-reviewed paper from Cloudflare

## ZKAttest from Cloudflare

The ZKAttest paper from Cloudflare presents an interesting solution to create ZKPs
on operations for the P-256 curve.
In short, they use a new elliptic curve with properties allowing it to create short
proofs on operations on the P-256 curve.
This new curve is introduced in the ZKAttest paper and called `T-256`, or `Tom-256`.
While the solution from Google is very generic and works even with an example
using mDoc, the `T-256` curve is only useful if the VC is based on BBS(+).

As with SD-JWT, the holder gets their VC from the issuer:

1. The holder proves their identity, creates a keypair with their Secure Element, and sends the public key to the issuer
2. The issuer creates a VC with all value of the fields hashed, including the public key from the holder
3. The issuer signs the VC using the its secret key, and sends it to the holder
4. The holder stores this VC in their device

Unlike the SD-JWT, the BBS VC can be 'blinded', but still be verified using the public
key from the issuer.

## Exercises

In this exercise, you'll set up a system with an issuer, a holder, and a verifier, and then
exchange messages between these actors.
The messages are kept as close as possible to a potential implementation, but are only provided
as an example of how holder binding using BBS can work.

## Rust

As this exercise is based on the excellent library from 
[docknetwork/crypto](https://github.com/docknetwork/crypto/tree/main/equality_across_groups),
all exercises are in rust.
Welcome to the future :)

This exercise tries to hide as much as possible regarding rust.
As such it is not very typical rust code, although it follows some best practices.
Under no circumstances should you use this code in production!
Here some guidelines:

- To create a new variable, use `let mut setup = PublicSetup::new();`.
The `mut` is necessary if the variable will be modified in the future. Different from Java- or Type-script, modifications include changing an array or a map!
- You can print variables using `println!("{setup}");`.
Depending on the variable type, this either prints a debugging representation, or a JSON.
- Rust is very jealous: if you want to pass the same variable more than once to a function,
you need to `clone()` it.
- Error handling is done with panics: if do something wrong, the code will panic.
- Look in the [ecdsa_proof](./ecdsa_proof/src/lib.rs) code for more documentation, or ask
one of the assistants!

## Documentation

You can find the full documentation here: [ecdsa_proof](./ecdsa_proof/target/doc/ecdsa_proof).
The following are the necessary methods for the exercises:

```rust
impl Issuer {
    /// Create a new issuer with a new keypair.
    pub fn new(mut setup: PublicSetup) -> Self;

    /// Returns the certificate of the [Issuer] - in this case the public key.
    pub fn get_certificate(&self) -> PublicKeyG2<Bls12_381>;

    /// Creates a new BBS credential using the coordinates of the holder's public key as the first two
    /// messages.
    /// The real issuer would request a proof that the holder has been authenticated and has the right
    /// to request a credential.
    /// In our case it adds a `first` and `last` to the messages.
    pub fn new_credential(&mut self, key_pub: SecP256Affine) -> VerifiedCredential;
}

impl MobilePhone {
    /// Creates a new mobile phone with an empty [SecureElement] and no [Swiyu] app
    /// installed.
    pub fn new(setup: PublicSetup) -> Self;

    /// Fills in the [Swiyu] app - engineer humor...
    pub fn install_swiyu(&mut self);

    /// Access the [SecureElement]. This is mostly to avoid explaining
    /// to students [Option]s and stuff.
    pub fn secure_element(&mut self) -> &mut SecureElement;

    /// Access the [Swiyu] app. This is mostly to avoid explaining
    /// to students [Option]s and stuff.
    /// This panics if the [Swiyu] app has not been installed.
    pub fn swiyu(&mut self) -> &mut Swiyu;
}

impl Verifier {
    /// A verifier needs the certificate of the [Issuer] to know how to verify incoming
    /// [BBSPresentation]s.
    pub fn new(setup: PublicSetup, certificate: PublicKeyG2<Bls12_381>) -> Self;

    /// Returns a time-based changing message which is used to avoid reply-attacks.
    pub fn create_message(&mut self) -> VerifierMessage;

    /// Checks that the given [BBSPresentation] and [ECDSAProof] correspond to the [VerifierMessage].
    /// It does the following checks:
    /// - is the [BBSPresentation] correctly signed by the [Issuer]?
    /// - does the [ECDSAProof] validate that the commitment to the public key in the [BBSPresentation]
    /// verifies the signature?
    pub fn check_proof(
        &self,
        verifier_message: VerifierMessage,
        bbs_presentation: BBSPresentation,
        ecdsa_proof: ECDSAProof,
    );
}

impl SecureElement {
    /// Creates a new keypair and stores the private key in the internal [Vec].
    /// It returns the key and an id.
    pub fn create_kp(&mut self) -> SEKeypair;

    /// Signs a message with the private key referenced by id.
    /// No checks are done to make sure this id exists or is tied to the current
    /// application.
    pub fn sign(&self, id: usize, msg: VerifierMessage) -> ecdsa::Signature;
}

impl Swiyu {
    /// Create a new [Swiyu] app. Of course the real app will do so much
    /// more to initialize itself. But here it's mostly a placeholder to
    /// show which actor does which parts.
    pub fn new(setup: PublicSetup) -> Self;

    /// Add a [VerifiedCredential].
    /// In this simple implementation, only one [VerifiedCredential] can exist.
    pub fn add_vc(&mut self, key_id: usize, credential: VerifiedCredential);

    /// Returns a [BBSPresentation] which is a proof that the holder knows the
    /// messages signed by the [Issuer].
    /// In addition to the normal protocol, this adds two [PublicCommitment]s
    /// to the proof, which hide the public key of the holder.
    /// When creating a [BBSPresentation], the [PublicCommitment]s are Open, meaning
    /// that their random values are accessible.
    /// Before sending a [BBSPresentation] to an untrusted entity, one must call
    /// [BBSPresentation::close()] to remove the random values.
    pub fn blinded_presentation(
        &mut self,
        message: VerifierMessage,
        reveal: Vec<usize>,
    );
}

/// A [SecureElement] keypair, with the private key stored in the
/// [SecureElement] only, and only accessible via its id.
#[derive(Debug)]
pub struct SEKeypair {
    pub id: usize,
    pub key_pub: SecP256Affine,
}

impl BBSPresentation {
    /// Close the commitments by removing the random values.
    pub fn close(self);

    /// Verify that:
    /// - the signature can be verified by the certificate
    /// - the signature matches the messages
    /// - the message_strings match the messages
    /// - the BBS signature matches the commitment_pub
    pub fn verify(
        &self,
        setup: PublicSetup,
        issuer_certificate: PublicKeyG2<Bls12_381>,
        verifier_message: &BlsFr,
    );
}
```

# 2.1 - Setting up the systems

In a first step, you need to create the three main actors of this exercise:

- `Issuer` represents the servers run by the Swiss government, and which can issue credentials representing a person.
For this exercise we don't consider other issuers, like ticket issuers, diploma issuers, or others.
The issuer has a certificate, its public key, which can be used by the verifier to check if the holder presents a
valid proof.
- `Holder` is the user who wants to participate in the E-ID system. Technically, the holder is represented by a mobile
phone, which has the `secure_element()` and the `swiyu()` application. The holder only has access to the public key
of the `secure_element()`, which makes all of this work necessary to prove they have a valid credential.
- `Verifier` requests a proof from the holder that they have been verified by the issuer. In this exercise we show
how this proof cannot been linked between two presentations, as all its fields are different.

A fourth element is already initialized: the `Setup` represents publicly known cryptographic data which must be shared
between the issuer, the holder, and the verifier. It needs to be passed to each of the three main actors, so that
they have the same setup.

All actors need to be declared as mutable, use `issuer`, `holder`, and `verifier` as names:

```rust
let mut issuer = ...
```

In [ ]:
:dep ./ecdsa_proof

use ecdsa_proof::*;

println!("Setting up systems");
let setup = PublicSetup::new();
let mut issuer = Issuer::new(setup.clone());
let mut holder = MobilePhone::new(setup.clone());
let mut verifier = Verifier::new(setup.clone(), issuer.get_certificate());

## Discussions

- What is in the `PublicSetup`, and which parts are created by whom?
- What can go wrong with this setup, and how is the system protected?
- Why does the verifier need the certificate?

# 2.2 - Installing the Swiyu app and adding a credential

Now we use the `holder` variable, which represents the mobile phone, and 'install' the Swiyu app.
The installation is done in a much more manual way than a standard user will be doing.
The goal here is to show the necessary steps and the data exchange between the different actors.

1. Install the Swiyu-app on the `holder`
2. Create a keypair from the `secure_element()`
3. Fetch a new credential from the `issuer`, passing it the public key from the secure element
4. Add the credential to the `swiyu()` app of the `holder`

In [ ]:
// Install the Swiyu app and add a credential
println!("Creating a credential");
holder.install_swiyu();
let secure_kp = holder.secure_element().create_kp();
let credential = issuer.new_credential(secure_kp.key_pub);
holder.swiyu().add_vc(secure_kp.id, credential);

## Discussions

- Where is the private key of the holder stored?
- What is stored in the credential sent by the issuer?
- Which part of the credential is usually presented to a verifier?

# 2.3 - Requesting a proof from the holder

Now that the holder has their credential, they are ready to use it.
For our very simple example, we suppose that a verifier wants to request information from a holder.
The verifier starts with a random message, to make sure that the proof cannot be used in another
context.
Here are the different steps necessary for the proof generation by the holder:

1. The `verifier` creates a new random message and sends it to the holder
2. Then the `holder` creates a signature on the `message` sent by the `verifier`
3. The `holder` uses their `swiyu()` to create a `blinded_presentation` of their credential
4. Finally the `holder` creates a proof that:
    - they know a public key `p` which can be used to verify the signature
    - this public key `p` is represented in the `blinded_presentation` of the credential
5. Don't forget to close the commitments of the `blinded_presentation`!

In [ ]:
// Verifier requests a presentation from the holder by creating a random message.
println!("Requesting a presentation");
let challenge = verifier.create_message();

// Holder creates a blinded presentation, an ECDSA signature, and a proof.
// This is of course all done in the Swiyu app, but here we do it step-by-step.
println!("Creating a presentation and an ECDSA proof");
let signature = holder.secure_element().sign(secure_kp.id, challenge.clone());
let presentation_open = holder.swiyu().blinded_presentation(challenge.clone(), vec![]);
let proof = ECDSAProof::new(setup.clone(), secure_kp.key_pub, credential.clone(), presentation_open.clone(), signature, challenge.clone());
let presentation = presentation_open.close();

## Discussions

- Which of the variables you created above should not be sent to the verifier? Why?
- How can you reveal one of the fields (there is a first- and a last-name stored in the credential)?
- Where do you see the revealed fields in the variables?
- Why do we need to 'close the commitment'? Why was it open?

# 2.4 - Verifying the proof

Now the `verifier` gets the proof, and can verify that:
- it is a valid signature by the `issuer`
- it has been signed by the public key hidden in the presentation

In [ ]:
// Holder sends the proof to the verifier, which checks it.
println!("Verifying the ECDSA proof");

verifier.check_proof(challenge.clone(), presentation.clone(), proof.clone());

println!("All good");

## Discussions

- Look at the slide 2.3 for proving a valid signature, and see if you can find in the
code where the verifications happen.
- What is the speed of the verification?

# 2.5 - More exercises

1) Verifying the signatures

  - Check that multiple presentations give unlinkable representations
  - Create another `verifier.create_message` and make sure that the verification fails
  - Create another `holder` and make sure that the different signature fails

2) Diving into the cryptography

  - The `BBSPresentation` structure has two fields for the commitments - what is the `Open` and `Close` state?
  - Look at the `ECDSAProof` structure to see which other commitment and which proofs are used
  - What are the verifications done in the `verifier`?
  - Which verification is still missing?

3) Revealing messages in the BBS Protocol

  - The `VerifiedCredential` has two fields which represent a `String`:
    - Change the second parameter to `blinded_presentation` by passing it one or both of
      - `VerifiedCredential::FIELD_FIRSTNAME`
      - `VerifiedCredential::FIELD_LASTNAME`
    - Then, after the `verifier.check_proof`, print the corresponding field using
      `presentation_closed.message_string`
  - Where is the verification made that these `String`s match?
  - What does it change with respect to the unlinkability to reveal data from the BBS Proof?